<a href="https://colab.research.google.com/github/atis06/tensorflow_bilstm_pooling/blob/master/Thesis_net_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/amirziai/cse6242-project/raw/master/resources/datasets/bbc-text.csv

--2020-02-14 09:36:32--  https://github.com/amirziai/cse6242-project/raw/master/resources/datasets/bbc-text.csv
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amirziai/cse6242-project/master/resources/datasets/bbc-text.csv [following]
--2020-02-14 09:36:33--  https://raw.githubusercontent.com/amirziai/cse6242-project/master/resources/datasets/bbc-text.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/plain]
Saving to: ‘bbc-text.csv.2’

bbc-text.csv.2      100%[===================>]   4.82M  --.-KB/s    in 0.04s   

2020-02-14 09:36:33 (123 MB/s) - ‘bbc-text.csv.2’ saved [50574

In [10]:


import csv
import tensorflow as tf
import numpy as np
import sklearn
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

print(tf.__version__)

vocab_size = 10000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = 'OOV'
training_portion = .8

articles = []
labels = []

with open("bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])



train_sequences = tokenizer.texts_to_sequences(train_articles)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_article(train_sequences[1]))
print(decode_article(train_padded[1]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
1.15.0
worldcom boss left books alone former worldcom boss bernie ebbers accused overseeing 11bn £5 8bn fraud never made accounting decisions witness told jurors david myers made comments questioning defence lawyers arguing mr ebbers responsible worldcom problems phone company collapsed 2002 prosecutors claim losses hidden protect firm shares mr myers already pleaded guilty fraud assisting prosecutors monday defence lawyer reid weingarten tried distance client allegations cross examination asked mr myers ever knew mr ebbers make accounting decision aware mr myers replied ever know mr ebbers make accounting entry worldcom books mr weingarten pressed replied witness mr myers admitted ordered false accounting entries request former worldcom chief financial officer scott sullivan defence lawyers trying paint mr sullivan admitted fraud testify later trial OOV behind worldco

In [0]:
#W2V
w2v_dim=300
import gensim
splitted_articles=[article.split() for article in articles]
model = gensim.models.Word2Vec(iter=5, size=w2v_dim, workers=4,min_count=4) 
model.build_vocab(splitted_articles)
model.train(splitted_articles, total_examples=len(splitted_articles), epochs = 15)

vocabulary_size = len(model.wv.vocab)


In [0]:
train_articles_w2v = []

for row in train_padded:
  w2v_sentence = []
  for word in enumerate(decode_article(row).split()):
    try:
      w2v_sentence.append(np.asarray(model.wv[word[1]]))
    except Exception as e:
      w2v_sentence.append(np.asarray(np.zeros(300)))
  train_articles_w2v.append(np.asarray(w2v_sentence))

#train_articles_w2v = np.asarray([elem[0:200] for elem in train_articles_w2v])

In [14]:
def next_batch(batch_size):
    idx = np.arange(0, len(train_articles_w2v))
    np.random.shuffle(idx)
    idx = idx[0:batch_size]
    data_shuffle = [train_articles_w2v[i] for i in idx]
    labels_shuffle = [training_label_seq[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


X_batch, y_batch = next_batch(40)
print(X_batch)

[[[ 8.62099648e-01 -7.58275092e-01 -5.10803342e-01 ...  1.10714602e+00
    7.02131867e-01  6.15425520e-02]
  [-1.09876436e-03  7.00126141e-02 -2.32930705e-01 ...  4.53263193e-01
    1.31419405e-01 -4.49505411e-02]
  [ 1.49697095e-01  3.25755291e-02 -1.69385076e-01 ...  2.96258718e-01
    2.56735742e-01 -1.86892077e-01]
  ...
  [ 7.75562525e-02 -3.08722910e-02 -1.60007313e-01 ...  2.17864543e-01
    2.28700489e-01  2.54304614e-02]
  [-1.74080038e+00  8.65813717e-02 -9.32320297e-01 ...  6.69865131e-01
    1.62692010e+00 -7.63924047e-02]
  [-1.49393833e+00  9.34744835e-01 -1.25198472e+00 ...  4.95028377e-01
    7.29091242e-02  1.04749000e+00]]

 [[ 6.97790533e-02  8.23096707e-02  3.71163222e-03 ...  1.75992966e-01
    8.14921334e-02 -5.02074957e-02]
  [-3.89044613e-01  1.26834655e+00 -6.07722163e-01 ... -6.90266192e-01
    1.29577294e-01 -1.95300543e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  ...
  [ 3.43776017e-01  2.2

In [0]:
validation_articles_w2v = []

for row in validation_padded:
  w2v_sentence = []
  for word in enumerate(decode_article(row).split()):
    try:
      w2v_sentence.append(np.asarray(model.wv[word[1]]))
    except Exception as e:
      w2v_sentence.append(np.asarray(np.zeros(300)))
  validation_articles_w2v.append(np.asarray(w2v_sentence))

#train_articles_w2v = np.asarray([elem[0:200] for elem in train_articles_w2v])

In [0]:
def print_shape(varname, var):
  with tf.Session() as sess:
    """
    :param varname: tensor name
    :param var: tensor variable
    """
    print('{0} : {1}'.format(varname, tf.shape(var, name=None)))

In [18]:
######################################################################


import tensorflow as tf

from tensorflow.keras.layers import LSTMCell, Bidirectional


# Just one feature, the time series(embeddig dim)
num_inputs = 300
# Num of steps in each batch (seqlength)
num_time_steps = 200
# 100 neuron layer, play with this
num_hidden = 200
# learning rate you can play with this
learning_rate = 0.001
# how many iterations to go through (training steps), you can play with this
epochs = 200
# Size of the batch of data
batch_size = 100
n_classes = 5
dropout_keep_prob = 0.8

'''with tf.Session() as sess:
    print(tf.one_hot(y_batch, n_classes,
                     axis=-1).eval())'''


def RNN(x, weights, biases, timesteps, num_hidden):
    x = tf.unstack(x, timesteps, 1)
    rnn_cell = tf.contrib.rnn.BasicRNNCell(num_hidden)
    states_series, current_state = tf.contrib.rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(states_series[-1], weights) + biases

def BiRNN(x, weights, biases, timesteps, num_hidden):

    #x = tf.unstack(x, timesteps, 1)
    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    if dropout_keep_prob is not None:
            lstm_fw_cell=tf.contrib.rnn.DropoutWrapper(lstm_fw_cell,output_keep_prob=dropout_keep_prob)
            lstm_bw_cell=tf.contrib.rnn.DropoutWrapper(lstm_bw_cell,output_keep_prob=dropout_keep_prob)
    outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                                 dtype=tf.float32)

    output_rnn=tf.concat(outputs,axis=2) #[batch_size,sequence_length,hidden_size*2]
    output_rnn_last=tf.reduce_max(output_rnn,axis=1) #[batch_size,hidden_size*2] # this is average pooling
    #output_rnn_last=output_rnn[:,-1,:] ##[batch_size,hidden_size*2] # this uses the last hidden state as the representation.
    print("output_rnn_last:", output_rnn_last)
    return tf.matmul(output_rnn_last, weights) + biases

tf.compat.v1.reset_default_graph()
X = tf.placeholder(tf.float32, [None, num_time_steps, num_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

W = tf.Variable(tf.random_normal(shape=[2*num_hidden, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))

output_logits = BiRNN(X, W, b, num_time_steps, num_hidden)
y_pred = tf.nn.softmax(output_logits)

cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
          X_batch, y_batch = next_batch(batch_size)
          X_batch = X_batch.reshape((batch_size, num_time_steps, num_inputs))
          y_batch = tf.one_hot(y_batch-1, n_classes, axis=-1).eval().reshape(-1, n_classes)

          sess.run(optimizer, feed_dict={X: X_batch, y: y_batch})
          if epoch % 100 == 0:
              mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
              print(epoch, "\tMSE:", mse)


    X_batch_valid = np.asarray(validation_articles_w2v).reshape((len(validation_articles_w2v), num_time_steps, num_inputs))
    y_batch_valid = tf.one_hot(validation_label_seq-1, n_classes, axis=-1).eval().reshape(-1, n_classes)

    feed_dict_valid = {X: X_batch_valid, y: y_batch_valid}

    loss_valid, acc_valid = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
    print('---------------------------------------------------------')
    print("Epoch: {0}, validation loss: {1:.2f}, validation accuracy: {2:.01%}".
          format(epoch + 1, loss_valid, acc_valid))
    print('---------------------------------------------------------')


    '''y_pred = sess.run(cls_prediction, feed_dict={X: X_batch_valid[0].reshape(-1,num_time_steps, num_inputs)})
    print(y_pred)
    index = tf.argmax(y_pred, axis=0).eval()
    print(index, y_batch_valid[0])'''


output_rnn_last: Tensor("Max:0", shape=(?, 400), dtype=float32)
0 	MSE: 6.1879497
100 	MSE: 0.008959124
---------------------------------------------------------
Epoch: 200, validation loss: 0.19, validation accuracy: 96.4%
---------------------------------------------------------


In [0]:
'''model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)